In [1]:
from src.Evaluation.Evaluator import EvaluatorHoldout
import scipy.sparse as sps
import random
import os
import numpy as np
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

URM_train = sps.load_npz('Data/URM_train.npz')
URM_test_complete = sps.load_npz('Data/URM_test.npz')

URM_all=URM_train + URM_test_complete

In [2]:
from src.Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])
def objective_function(trial):
    # Parametri per early stopping
    early_stopping_params = {
        "validation_every_n": 1,
        "stop_on_validation": True,
        "evaluator_object": evaluator,
        "lower_validations_allowed": 15,
        "validation_metric": "MAP",
        "epochs": 500
    }

    recommender_instance = IALSRecommender(URM_train)
    recommender_instance.fit(
        num_factors = trial.suggest_int("num_factors", 1, 500, log=True),
        confidence_scaling = trial.suggest_categorical("confidence_scaling", ["linear", "log"]),
        alpha = trial.suggest_float("alpha", 0.1, 10.0, log=True),
        epsilon = trial.suggest_float("epsilon", 1e-3, 1.0, log=True),
        reg = trial.suggest_float("reg", 1e-6, 1e-2, log=True),
        init_mean = trial.suggest_float("init_mean", -0.1, 0.1),
        init_std = trial.suggest_float("init_std", 0.01, 0.1),
        **early_stopping_params
    )

    epochs = recommender_instance.get_early_stopping_final_epochs_dict()["epochs"]
    trial.set_user_attr("epochs", epochs)

    result_df, _ = evaluator.evaluateRecommender(recommender_instance)

    return result_df.loc[10, "MAP"]

ModuleNotFoundError: No module named 'Recommenders'

In [ ]:
import optuna

from optuna.samplers import RandomSampler

database_path='hypertuning_results/tuning_IALS.db'

study = optuna.create_study(direction='maximize', study_name='hyperparameters_IALS',
                                storage=f'sqlite:///{database_path}', load_if_exists=True,
    sampler=RandomSampler(seed=SEED))

study.optimize(objective_function, n_trials= 60,n_jobs=4)